In [ ]:
import os
import numpy as np
import pandas as pd
import gc

### If you found this helpful Do UpVote / Comment please.

# Combined Customer Data with Articles data basesd on article_id

In [ ]:
df = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv', chunksize=100000)
articles = pd.read_csv('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
users = next(df)
df = users.merge(articles, on='article_id')

In [ ]:
df['t_dat'] = pd.to_datetime(df['t_dat'], format="%Y-%m-%d")

In [ ]:
import numpy as np
df['timestamp'] = df.t_dat.values.astype(np.int64) // 10 ** 9


In [ ]:
df

In [ ]:
df_ar = pd.read_csv(r"/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv", dtype={'article_id': 'str'})
df_ar.head()

# Total Articles

In [ ]:
for col in df_ar.columns:
    print(col,len(pd.unique(df_ar[col])))
    

# Unique Customers & Transactions from combied DataFrame ( ChunkSize 100000 )

In [ ]:
for col in df.columns:
    print(col,len(pd.unique(df[col])))
    

## Reduced multicollinearity from articles data
Several columns are [category_text, encoded_value]. Thus, in order to eliminate multicollinearity, only one column will be included in each couple
Following are a couple of columns in item features, and we will keep one of them:

* use product_type_no - skip product_type_name
* use graphical_appearance_no - skip graphical_appearance_name
* use colour_group_code - skip colour_group_name
* use perceived_colour_value_id - skip perceived_colour_value_name
* use perceived_colour_master_id - skip perceived_colour_master_name
* use index_code - skip index_name
* use index_group_no - skip index_group_name
* use section_no - skip section_name
* use garment_group_no - skip garment_group_name
* use product_code, skip product_name
* use department_no, skip department_name

In [ ]:
df_ar = df_ar.drop(columns = ['product_type_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name',
                        'perceived_colour_master_name', 'index_name', 'index_group_name', 'section_name', 
                        'garment_group_name', 'prod_name', 'department_name', 'detail_desc'])
df_ar.head()

In [ ]:
!mkdir /kaggle/working/recbox_data
df_ar.to_csv(r'/kaggle/working/recbox_data/recbox_data.item', index=False, sep='\t')

In [ ]:
temp = df[df['timestamp'] < 1585620000][['customer_id', 'article_id', 'timestamp']].rename(
    columns={'customer_id': 'user_id:token', 'article_id': 'item_id:token', 'timestamp': 'timestamp:float'})
temp

In [ ]:
df.to_csv('/kaggle/working/recbox_data/recbox_data.inter', index=False, sep='\t')

gc.collect()

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
sub0 = pd.read_csv('../input/hm-pre-recommendation/submissio_byfone_chris.csv').sort_values('customer_id').reset_index(drop=True)
sub1 = pd.read_csv('../input/hm-pre-recommendation/submission_trending.csv').sort_values('customer_id').reset_index(drop=True)
sub2 = pd.read_csv('../input/hm-pre-recommendation/submission_exponential_decay.csv').sort_values('customer_id').reset_index(drop=True)

sub0.shape, sub1.shape, sub2.shape

# Exploring PySpark

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from pyspark.sql import SQLContext 
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import udf,col,when
from pyspark.sql.functions import to_timestamp,date_format
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

sc = SparkSession.builder.appName("Recommendations").config("spark.sql.files.maxPartitionBytes", 5000000).getOrCreate()
spark = SparkSession(sc)

In [ ]:
transaction = spark.read.option("header",True) \
              .csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
transaction.printSchema()

In [ ]:
article = spark.read.option("header",True) \
              .csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv")
article.printSchema()

## Duration of transaction

In [ ]:
from pyspark.sql.functions import min, max
from pyspark.sql.functions import unix_timestamp, lit
min_date, max_date = transaction.select(min("t_dat"), max("t_dat")).first()
min_date, max_date

In this transaction dataset we have 31,788,324 rows and 5 columns.Let's capture first what are the most recently bought articles.For recommendation I am selecting only date 2020-09-22 which is the last transaction date.

In [ ]:
hm =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
hm = hm.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
hm = hm.withColumn('year', year(col('date')))
hm = hm.withColumn('month', month(col('date')))
hm = hm.withColumn('day', date_format(col('date'), "d"))

hm = hm[hm['year'] == 2020]
hm = hm[hm['month'] == 9]
hm = hm[hm['day'] == 22]
transaction.unpersist()

# Prepare the dataset
hm = hm.groupby('customer_id', 'article_id').count()
hm.show(5)

In [ ]:
hm =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
hm = hm.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
hm = hm.withColumn('year', year(col('date')))
hm = hm.withColumn('month', month(col('date')))
hm = hm.withColumn('day', date_format(col('date'), "d"))

transaction.unpersist()

# Prepare the dataset
hm = hm.groupby('customer_id', 'article_id').count()
hm.show(5)

In [ ]:
print((hm.count(), len(hm.columns)))

In [ ]:
# Count the total number of article count in the dataset
numerator = hm.select("count").count()

# Count the number of distinct customerid and distinct articleid
num_users = hm.select("customer_id").distinct().count()
num_articles = hm.select("article_id").distinct().count()

# Set the denominator equal to the number of customer multiplied by the number of articles
denominator = num_users * num_articles

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("Sparsity: ", "%.2f" % sparsity + "%.")

In [ ]:
userId_count = hm.groupBy("customer_id").count().orderBy('count', ascending=False)
userId_count.show()

In [ ]:
articleId_count = hm.groupBy("article_id").count().orderBy('count', ascending=False)
articleId_count.show()

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [ ]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(hm.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(hm).transform(hm)
transformed.show()

In [ ]:
(training,test)=transformed.randomSplit([0.8, 0.2])

To build the model explicitly specify the columns. Set nonnegative as ‘True’, since we are looking count greater than 0. The model also gives an option to select implicit ratings. Since we are working with explicit, set it to ‘False’ or by default it takes explicit.

When using simple random splits as in Spark’s CrossValidator or TrainValidationSplit, it is actually very common to encounter users and/or items in the evaluation set that are not in the training set. By default, Spark assigns NaN predictions during ALSModel.transform when a user and/or item factor is not present in the model.We set cold start strategy to ‘drop’ to ensure we don’t get NaN evaluation metrics.

In [ ]:
als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

In [ ]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="count",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)

In [ ]:
print("RMSE="+str(rmse))

In [ ]:
predictions.show()

In [ ]:
user_recs=model.recommendForAllItems(10).show(10)

In [ ]:
item_recs=model.recommendForAllUsers(10).show(10)

In [ ]:
%%time
userRecsDf = model.recommendForAllUsers(10).cache()
userRecsDf.count()

In [ ]:
userRecsDf.printSchema()

In [ ]:
userRecsDf.select("customer_id_index","recommendations.article_id_index").show(10,False)

In [ ]:
import gc #This is to free up the memory
gc.collect()

As seen in above image the results are in integer form we need to convert it back to its original name.The code is little bit longer given so many conversions

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
import pandas as pd
recs=model.recommendForAllUsers(10).toPandas()
nrecs=recs.recommendations.apply(pd.Series) \
            .merge(recs, right_index = True, left_index = True) \
            .drop(["recommendations"], axis = 1) \
            .melt(id_vars = ['customer_id_index'], value_name = "recommendations") \
            .drop("variable", axis = 1) \
            .dropna() 
nrecs=nrecs.sort_values('customer_id_index')
nrecs=pd.concat([nrecs['recommendations'].apply(pd.Series), nrecs['customer_id_index']], axis = 1)

In [ ]:
nrecs.columns = ['ArticleID_index','count','UserID_index']
md=transformed.select(transformed['article_id'],transformed['article_id_index'],transformed['customer_id'],transformed['customer_id_index'])
md=md.toPandas()

In [ ]:
dict1 =dict(zip(md['article_id_index'],md['article_id']))
dict2=dict(zip(md['customer_id_index'],md['customer_id']))
nrecs['article_id']=nrecs['ArticleID_index'].map(dict1)
nrecs['customer_id']=nrecs['UserID_index'].map(dict2)